In [118]:
pip install kneed

Note: you may need to restart the kernel to use updated packages.


In [119]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [120]:
import torch

In [121]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random
import pandas as pd
import warnings

import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
import itertools
from tqdm import tqdm
import seaborn as sns
from kneed import KneeLocator
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean

## Part 1: Data Preparation 
### Adjust the bond prices to mitigate the impact of the benchmark bond roll.

In [122]:
tr_dm = pd.read_excel('GENERIC BOND PRICE.xlsx', sheet_name='DM_PRICE')
tr_dm.Dates = pd.to_datetime(tr_dm.Dates)
tr_dm = tr_dm.set_index('Dates')

In [123]:
def calculate_dirty_price(clean_price, coupon, days_since_last_coupon):
    daily_coupon = coupon / 365  
    accrued_interest = daily_coupon * days_since_last_coupon
    return clean_price + accrued_interest

def calculate_days_since_last_coupon(date, last_coupon_date):
    return (date - last_coupon_date).days

def find_first_coupon_date(coupon_series):
    coupon_changes = coupon_series.diff().dropna()
    if len(coupon_changes) > 0:
        first_change_date = coupon_changes.index[0]
        return first_change_date - pd.Timedelta(days=180)  # Assume coupon paid 6 months before the change
    else:
        return coupon_series.index[0] 
    

In [124]:
dirty_price_series = {}
for column in tr_dm.columns:
    if column.endswith('Govt'):
        clean_price_series = tr_dm[column]
        coupon_series = tr_dm[f"{column} CPN"]

        first_coupon_date = find_first_coupon_date(coupon_series)
        last_coupon_date = first_coupon_date
        current_coupon = coupon_series.iloc[0]
        
        dirty_prices = []
        
        for date, clean_price in clean_price_series.items():
            if coupon_series[date] != current_coupon:
                last_coupon_date = date - pd.Timedelta(days=180)
                current_coupon = coupon_series[date]
            
            days_since_last_coupon = calculate_days_since_last_coupon(date, last_coupon_date)
            
            dirty_price = calculate_dirty_price(clean_price, current_coupon, days_since_last_coupon)
            dirty_prices.append(dirty_price)

            if days_since_last_coupon >= 180:
                last_coupon_date = date

        tr_dm[f"{column}_Dirty"] = pd.Series(dirty_prices, index=clean_price_series.index)

In [125]:

for bond_col in tr_dm.columns:
    if bond_col.endswith('Dirty'): 
        coupon_col = bond_col.replace('Govt_Dirty', 'Govt CPN')
        bond_col = bond_col.replace('_Dirty','')
        
        tr_dm[f"{bond_col} Adjusted"] = tr_dm[bond_col]
        
        coupon_changes = tr_dm[coupon_col].diff().fillna(0) != 0

        for change_date in tr_dm.index[coupon_changes]:

            price_on_change = tr_dm.loc[change_date, bond_col]
            
            previous_price = tr_dm.loc[tr_dm.index[tr_dm.index.get_loc(change_date) - 1], bond_col]
            
            price_diff = price_on_change - previous_price
            
            tr_dm.loc[change_date:, f"{bond_col} Adjusted"] -= price_diff

        tr_dm[f"{bond_col} Adjusted Return"] = tr_dm[f"{bond_col} Adjusted"].pct_change()


# col_adj = [i for i in tr_dm.columns if (i.endswith('Adjusted') or i.endswith('Adjusted Return'))]

col_adj_price = [i for i in tr_dm.columns if i.endswith('Adjusted')]

col_adj_return = [i for i in tr_dm.columns if i.endswith('Adjusted Return')]

tr_dm_net = tr_dm[col_adj_return].fillna(0)

tr_dm_net_price = tr_dm[col_adj_price].fillna(0)

## Part 3: Identify Correlated Assets (Developed Market)
### Step1 : PCA (cluster by the first principal component)

In [126]:
from statsmodels.tsa.stattools import coint

def test_cointegration_in_clusters(data, cluster_dict):

    cointegrated_pairs = []
    for cluster_num in cluster_dict:
        asset_names = cluster_dict[cluster_num]
        asset_names = [i.replace(' Return', '') for i in asset_names]
        
        # Loop through each pair of assets in the cluster
        for i in range(len(asset_names)):
            for j in range(i + 1, len(asset_names)):
                asset1 = asset_names[i]
                asset2 = asset_names[j]
             
                series1 = data[asset1]
                series2 = data[asset2]
                
                # Perform the Engle-Granger cointegration test
                coint_t, p_value, _ = coint(series1, series2)
                
                # set a higher significant level (0.2) to avoid missing potential relationship
                if p_value < 0.2:  
                    cointegrated_pairs.append([asset1,asset2])
                else:
                    pass
                    # print(f"  {asset1} and {asset2} are NOT cointegrated (p-value: {p_value:.4f})")

    return cointegrated_pairs


In [127]:

def noncoherent_pair_cluster(data, data_price): # input: return data, price data
    # Step 1: Cluster by Principal component 1
    scaler = StandardScaler()
    asset_returns = pd.DataFrame(scaler.fit_transform(data), columns= data.columns)


    # Calculate the loadings of bond returns on PCs
    K = 1
    pca = PCA(n_components=K)
    pca.fit(asset_returns)
    loadings = pca.components_.T

    wcss = []
    for k in range(1, 10):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(loadings)
        # Inertia: Sum of squared distances to closest cluster center
        wcss.append(kmeans.inertia_)  
        
    # Use the KneeLocator to detect the elbow point
    kneedle = KneeLocator(range(1, 10), wcss, S=1.0, curve='convex', direction='decreasing')

    # Get the optimal number of clusters
    optimal_clusters = kneedle.elbow


    # Clustering in the principal component space and using K-means to cluster different govt bonds
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    clusters = kmeans.fit_predict(loadings)
    asset_names = asset_returns.columns

    cluster_dic = {}
    for cluster in range(optimal_clusters):
        cluster_assets = asset_names[clusters == cluster]
        cluster_dic[cluster + 1] =cluster_assets


    # Step 2: Find cointegration pairs
    cointegrated_pairs = test_cointegration_in_clusters(data_price, cluster_dic)

    # Step 3: Exclude pairs have similar PC2 and PC3
    K = 3
    pca = PCA(n_components=K)
    pca.fit(asset_returns)
    loadings = pca.components_.T

    loadings_df = pd.DataFrame(loadings, columns = ['PC1','PC2','PC3'], index = data.columns)

    threshold_pc2 = 0.01 
    threshold_pc3 = 0.01

    coherent_pair = []
    noncoherent_pair = []

    for i in range(len(cointegrated_pairs)):
        asset1 = cointegrated_pairs[i][0]
        asset2 = cointegrated_pairs[i][1]
        
        bond1_loadings_pc2 = loadings_df.loc[f"{asset1} Return", 'PC2'] 
        bond2_loadings_pc2 = loadings_df.loc[f"{asset2} Return", 'PC2'] 
        
        bond1_loadings_pc3 = loadings_df.loc[f"{asset1} Return", 'PC3']
        bond2_loadings_pc3 = loadings_df.loc[f"{asset2} Return", 'PC3']  
        
        # Euclidean distance between PC2 and PC3 loadings
        distance_pc2 = euclidean([bond1_loadings_pc2], [bond2_loadings_pc2])
        distance_pc3 = euclidean([bond1_loadings_pc3], [bond2_loadings_pc3])
        
        if distance_pc2 > threshold_pc2 and distance_pc3 > threshold_pc3:
            noncoherent_pair.append((asset1, asset2))
        else:
            coherent_pair.append((asset1, asset2))
            

    return noncoherent_pair


## Part 4: Naiive Rule-based Trading Algorithm

The code below implements a rule-based pair trading strategy that is based on the 2nd principal component loadings of the asset pair to construct a market-neutral spread.

Key Steps:
1. Rolling PCA Calculation:

Option 1: PCA is applied on the returns of only two selected assets. In this case, the first principal component (PC1) represents the common trend between the two selected assets.

Option 2: PCA can be applied to all 11 bonds, which would make the PC1 represent the broader market trend for all treasury bonds. PC2 represents the specific factors that diverge from the broader market, allowing for more targeted trading opportunities.

2. Rolling Loadings:

The PCA loadings are calculated on a rolling basis to avoid look-ahead bias over time. The strategy focuses on the loadings for the second principal component (PC2), which reflects the bond-specific factors that are relatively insensitive (orthogonal) to broader market movements captured by PC1.
The code extracts the rolling loadings of the selected assets on PC2 and uses these loadings to construct the spread.

3. Spread Construction:

The spread is calculated as a linear combination of the two assets' returns weighted by their PC2 loadings.
Then the weights are normalized based on the total absolute weight of the two assets to ensure that the portfolio remains balanced.

$ \text{Spread}  = w1 * \text{Asset 1 Return} - w2 * \text{Asset 2 Return}$

$ w1 = 1 $


$ w2 = w1 * \frac{\sigma 1}{\sigma 2} * \frac{PC loading Asset 1}{PC Loading Asset 2} $



1. Trading Signals:

Buy Signal: Generated when the z-score of the spread (number of standard deviations the spread deviates from its rolling mean) falls below a specified lower threshold. This implies the spread has diverged significantly, and the strategy goes long on the spread, expecting a reversal.
Sell Signal: Generated when the z-score rises above the upper threshold, indicating the spread has widened significantly. The strategy goes short on the spread, expecting a convergence.
Exit Signal: The strategy closes the positions when the spread reverts to a level closer to the mean, as defined by a close threshold.

5. Z-Score and Thresholds:

The z-score is calculated based on the rolling mean and rolling standard deviation of the spread. This z-score is used to quantify the divergence of the spread from its historical average.
The thresholds (upper, lower, and close) dictate when the strategy enters and exits positions.

6. Performance Evaluation:

The code tracks the cumulative returns of the pair trading strategy over time. It shifts the positions by one day to avoid look-ahead bias when calculating the returns for the next day.
The final cumulative returns are plotted to visualize the strategy's performance for the selected asset pair.

In [128]:
def rolling_pca_loadings(data, window_size, num_components=2):
    rolling_loadings = []
    
    for i in range(window_size, len(data) + 1):
        window_data = data[i - window_size:i]
        pca = PCA(n_components=num_components)
        pca.fit(window_data)
        loadings = pca.components_.T  
        rolling_loadings.append(loadings)
        
    return np.array(rolling_loadings)

def diff_sign_pc_loadings(data, window_size):
    loadings = rolling_pca_loadings(data, window_size, num_components=2)
    result = []
    for matrix in loadings:
        first_col = matrix[:, 0]
        second_col = matrix[:, 1]
        
        if np.sign(first_col[0]) == np.sign(first_col[1]):
            result.append(first_col)
        else:
            result.append(second_col)
    
    result_array = np.array(result)
    
    return result_array

In [129]:
# Non-coherent pair
def rule_based_strategy(data, window_size, upper_threshold,close_threshold, asset1,asset2):

    # 1. Rolling PCA Calculation:
    # option1: apply pca on two assets: pc1 represents the common trend between asset1 and asset 2 only
    selected_asset_returns = data[[f'{asset1}', f'{asset2}']]
    
    # 2. Rolling Loadings:
    rolling_loadings = diff_sign_pc_loadings(selected_asset_returns, window_size)

    # rolling_pc1_loadings = rolling_loadings[:, :, 0]  # First component loadings
    # rolling_pc2_loadings = rolling_loadings[:, :, 1]  # Second component loadings
    
    
    pc_loading_asset1 = rolling_loadings[:, 0] 
    pc_loading_asset2 = rolling_loadings[:, 1] 

    # 3. Spread Construction:
    
    sigma_asset1 = data[f'{asset1}'].rolling(window=window_size).std().dropna()
    sigma_asset2 = data[f'{asset2}'].rolling(window=window_size).std().dropna()
    pc_loading_asset1 = pd.Series(pc_loading_asset1, index = sigma_asset1.index)
    pc_loading_asset2 = pd.Series(pc_loading_asset2, index = sigma_asset2.index)

    sigma_ratio = sigma_asset1 / sigma_asset2
    
    w1 = 1
    w2 = (w1 * sigma_ratio * pc_loading_asset1)/pc_loading_asset2
    
    spread = w1 * data[f'{asset1} Return'] - w2 * data[f'{asset2} Return']

    # 4. Trading Signals:
    rolling_mean = spread.rolling(window=window_size).mean()
    rolling_std = spread.rolling(window=window_size).std()

    # 5. Z-Score and Thresholds:
    z_score = (spread - rolling_mean) / rolling_std
    
    lower_threshold = - upper_threshold
    
    positions = pd.DataFrame(index=data.index, columns=['Position','Holdings_w1','Holdings_w2'])
    
    # Enter signal (spread deviates from the mean)
    positions['Position'] = np.where(z_score > upper_threshold, -1, 0)
    positions['Position'] = np.where(z_score < lower_threshold, 1, positions['Position'])
    
    # Exit signal (spread reverts to the mean)
    positions['Position'] = np.where((z_score > - close_threshold) & (positions['Position'] == 1), 0, positions['Position'])
    positions['Position'] = np.where((z_score < close_threshold) & (positions['Position'] == -1), 0, positions['Position'])
    
    positions['Holdings_w1'] = positions['Position'] * w1
    positions['Holdings_w2'] = positions['Position'] * w2


    # 6. Performance Evaluation:
    positions_shifted = positions.shift(1)  
    returns = positions_shifted['Holdings_w1'] * data[f'{asset1}'] + positions_shifted['Holdings_w2'] * data[f'{asset2}']
    initial_investment = 1
    cumulative_returns = (1+ returns).cumprod() -1

    return cumulative_returns[-1]

### Grid search for the optimal parameters

In [130]:
def optimize_parameters(data, noncoherent_pair, window_size_list, upper_threshold_list, close_threshold_list):

    param_results = {}
    pair_results = {}
    
    # Try each parameter combination
    for window_size, upper_threshold, close_threshold in itertools.product(
            window_size_list, upper_threshold_list, close_threshold_list):
        
        if close_threshold >= upper_threshold:
            continue
            
        returns = []
        pair_details = []
        
        for asset_pair in noncoherent_pair:
            asset1 = asset_pair[0]#.replace(' Adjusted Return', '')
            asset2 = asset_pair[1]#.replace(' Adjusted Return', '')
            
            # Get return for this pair with current parameters
            result = rule_based_strategy(data, window_size, upper_threshold, close_threshold, asset1, asset2)
            returns.append(result)
            
            pair_details.append({'asset1': asset1, 'asset2': asset2, 'return': result})
        
        # Store average return for this parameter combination
        param_key = (window_size, upper_threshold, close_threshold)
        param_results[param_key] = {'avg_return': np.mean(returns),'pair_details': pair_details}

    
    # Find best parameter combination
    best_params = max(param_results.items(), key=lambda x: x[1]['avg_return'])
    window_size, upper_threshold, close_threshold = best_params[0]
    
    
    return upper_threshold, close_threshold, window_size


## Part 5: Reinforcement Learning Based Strategy

To enhance the existing rule-based strategy for pair trading using a Deep Q-Network (DQN), we can replace the rule-based signals with a reinforcement learning (RL) approach. The purpose for RL step is to find 'when' to trade and 'how much' to trade by interacting with observation space, action space and reward space.

Key Points:

Teachnique: DQN Strategy

Dynamic Pair Selection & Training Process
1. Formation Period (Year T): Select correlated pairs using PCA
2. Training Period (Year T+1): Train RL agent on selected pairs
3. Testing Period (Year T+2): Test agent on newly generated pairs
4. Repeat process by rolling forward one year for robust validation

![image.png](attachment:./attachment:853841c4-393a-4b7c-870b-290257aa81f2.png)

Model Architecture
1. Deep Q-Network (DQN) for automated trading decisions 
2. State space includes normalized spread, z-score, position metrics
3. Action space: Long (1), Neutral (0), Short (-1) positions
4. Reward = Return - beta * (RL action - rule-based strategy action)

Training Process:
1. the agent learns optimal entry and exit points through extensive episodes with different pairs and market conditions
2. Set the network with highest reward as target network, agent continuously learns to match or exceed this taret network performance
3. the agent develop resilience to both regime changes and switching between different asset pairs, making it truly adaptive to market dynamics.

In [131]:
# defines a new class called DQN that inherits from PyTorch's nn.Module. 

import torch
import torch.nn as nn
import torch.nn.functional as F

# todo: to be fine-tuned
class DQN(nn.Module):
    def __init__(self, state_size, action_size,activation, hidden_sizes=[128,128], dropout_rate=0.2):
        super(DQN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if activation == 'relu':
            act_fn = nn.ReLU()
        elif activation == 'leaky_relu':
            act_fn = nn.LeakyReLU(0.01)
        elif activation == 'elu':
            act_fn = nn.ELU()
        elif activation == 'selu':
            act_fn = nn.SELU()
        elif activation == 'tanh':
            act_fn = nn.Tanh()

    
        # Input layer
        self.layers.append(nn.Linear(state_size, hidden_sizes[0]))
        self.layers.append(nn.LayerNorm(hidden_sizes[0]))
        self.layers.append(act_fn)
        self.layers.append(nn.Dropout(dropout_rate))
        
        # Hidden layers
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.layers.append(nn.LayerNorm(hidden_sizes[i]))
            self.layers.append(act_fn)
            self.layers.append(nn.Dropout(dropout_rate))
        
        # Output layer
        self.layers.append(nn.Linear(hidden_sizes[-1], action_size))

    def forward(self, state):
        x = state
        for layer in self.layers:
            x = layer(x)
        return x
    
# creates a named tuple to store experience replays. a convenient way to group related data.
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

In [132]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [133]:
class DQNAgent:
    def __init__(self, state_size, action_size,active_func, learning_rate=1e-3):
        self.active_func= active_func
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer(10000) # a replay buffer with a capacity of 10,000 experiences
        self.gamma = 0.95    # discount factor
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = learning_rate
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = DQN(state_size, action_size,active_func).to(self.device)
        self.target_net = DQN(state_size, action_size,active_func).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        self.batch_size = 32

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate) #update for policy network's weight
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='max', 
                                                            factor=0.1, patience=10)
        
    def act(self, state):
        if random.random() <= self.epsilon: # exploration (random act)
            return random.randrange(self.action_size)
        
        with torch.no_grad(): # exploitation: follow policy network
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)  
            action_values = self.policy_net(state)
            return action_values.max(1)[1].item()

    def step(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done) # add new experience to replay buffer

    def learn(self):

        if len(self.memory) < self.batch_size: # check if enough space in buffer
            return
        
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.FloatTensor(batch.state).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(self.device)
        
        # pass our batch of states through our policy network to get Q-values for all actions. 
        # use gather to select only the Q-values for the actions that were actually taken. This gives us our current estimate of the Q-values for our sampled state-action pairs
        current_q_values = self.policy_net(state_batch).gather(1, action_batch)

        # For each non-terminal next state, we compute the maximum Q-value using our target network. 
        # We use the target network (which is updated less frequently) to provide a more stable target for learning.
        next_states = torch.FloatTensor([s for s in batch.next_state if s is not None]).to(self.device)
        with torch.no_grad():
            next_actions = self.policy_net(next_states).max(1)[1]
            next_state_values = self.target_net(next_states).gather(1, next_actions.unsqueeze(1))

        # expected (target) Q-values using the Bellman equation.
        expected_q_values = (next_state_values * self.gamma) + reward_batch
        
        # Weighted loss
        loss = F.smooth_l1_loss(current_q_values, expected_q_values)

        # compute the gradients of the loss with respect to our network parameters, and then update our network parameters (weights and bias) to minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    # every 10 episode, we will copy weights from training policy network to target network
    def update_target_net(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    # as we have more training samples, we don't need to explore too often
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


In [134]:
class MultiPairTradingEnv:
    def __init__(self, asset_returns_raw, pair_list, upper_threshold, close_threshold, window_size, beta):
        self.asset_returns_raw = asset_returns_raw
        self.pair_list = pair_list  
        self.window_size = window_size
        self.upper_threshold = upper_threshold
        self.close_threshold = close_threshold
        self.beta = beta
        self.action_space = [-1, 0, 1]
        
        # Initialize spreads for all pairs
        self.spreads = {}
        self.w2s = {}
        for asset1, asset2 in pair_list:
            spread, w2 = self._calculate_spread(asset1, asset2)
            self.spreads[(asset1, asset2)] = spread
            self.w2s[(asset1, asset2)] = w2
        
        # Initialize training variables
        self.reset()

    def _calculate_spread(self, asset1, asset2):
        """Calculate spread for a given pair"""
        selected_asset_returns = self.asset_returns_raw[[f'{asset1} Return', f'{asset2} Return']]
        
        rolling_loadings = diff_sign_pc_loadings(selected_asset_returns, self.window_size)
        pc_loading_asset1 = rolling_loadings[:, 0]
        pc_loading_asset2 = rolling_loadings[:, 1]
        
        sigma_asset1 = self.asset_returns_raw[f'{asset1} Return'].rolling(window=self.window_size).std().dropna()
        sigma_asset2 = self.asset_returns_raw[f'{asset2} Return'].rolling(window=self.window_size).std().dropna()
        
        pc_loading_asset1 = pd.Series(pc_loading_asset1, index=sigma_asset1.index)
        pc_loading_asset2 = pd.Series(pc_loading_asset2, index=sigma_asset2.index)
        sigma_ratio = sigma_asset1 / sigma_asset2
        
        w1 = 1
        w2 = (w1 * sigma_ratio * pc_loading_asset2) / pc_loading_asset1
        total_weight = abs(w1) + abs(w2)
        w1 = w1 / total_weight
        w2 = w2 / total_weight

        spread = w1 * self.asset_returns_raw[f'{asset1} Return'] - w2 * self.asset_returns_raw[f'{asset2} Return']
        
        return spread, w2

    def reset(self,ifbacktest = False, pair = []):
        # Randomly select a pair
        if ifbacktest:
            self.current_pair = pair
        else:
            self.current_pair = random.choice(self.pair_list)
        self.asset1, self.asset2 = self.current_pair
        
        # Get spread and w2 for selected pair
        self.spread = self.spreads[self.current_pair]
        self.w2 = self.w2s[self.current_pair]
        
        # Reset position and portfolio
        self.position = 0
        self.previous_position = 0
        self.portfolio_value = 1
        self.steps_taken = 0
        self.cumulative_return = [self.portfolio_value]
        # Reset tracking variables
        self.portfolio_start_value = self.portfolio_value
        self.last_position_change = None
        self.position_start_value = None
        self.return_list = []
        self.action_history = []
        self.reward = 0
        
        self.current_step = self.window_size
        
        return self._get_normalized_state()

    def step(self, action):
        self.steps_taken += 1
        self.previous_position = self.position
        new_position = self.action_space[action]
        
        # Handle position changes
        portfolio_reward = 0
        if new_position != 0 and self.previous_position == 0:
            self.last_position_change = self.current_step
            self.position_start_value = self.portfolio_value
        elif new_position == 0 and self.previous_position != 0:
            self.last_position_change = None
            self.position_start_value = None
            portfolio_reward = self._calculate_portfolio_reward()

        self.position = new_position
        
        # Calculate rewards
        spread_return = self._calculate_immediate_reward()
        baseline_action = self._get_baseline_action()
        deviation_penalty = self.beta * abs(action - baseline_action)
        self.reward = portfolio_reward - deviation_penalty

        # Update portfolio value
        self.return_list.append(1 + spread_return)
        self.cumulative_return = np.cumprod(self.return_list) -1
        self.portfolio_value = self.cumulative_return[-1]
        
        # Store action
        self.action_history.append({
            'step': self.current_step,
            'pair': self.current_pair,
            'position': self.position,
            'reward': self.reward,
            'spread_return': spread_return,
            'portfolio_value': self.portfolio_value,
            'cumulative_return': self.cumulative_return
        })
        
        # Advance step
        self.current_step += 1
        
        # Episode ends after 1000 steps or at end of data
        done = self.current_step >= len(self.spread) - 1
        
        # Close position at end of episode
        if done and self.position != 0:
            self.position = 0
            portfolio_reward = self._calculate_portfolio_reward()
            self.reward += portfolio_reward
        
        return self._get_normalized_state(), self.reward, done, {
            'pair': self.current_pair,
            'portfolio_value': self.portfolio_value,
            'spread_return': spread_return,
            'deviation_penalty': deviation_penalty
        }

    def _get_normalized_state(self):
        """Get normalized state representation"""
        spread = self.spread.iloc[self.current_step]
        z_score = self._calculate_z_score(spread)
        
        # Normalize spread using recent window
        window = self.spread.iloc[max(0, self.current_step-100):self.current_step+1]
        norm_spread = spread / np.std(window)
        
        # Normalize position duration and profit
        position_duration = 0 if self.last_position_change is None else self.current_step - self.last_position_change
        norm_duration = np.clip(position_duration / 100, 0, 1)
        
        position_profit = 0 if self.position_start_value is None else (self.portfolio_value - self.position_start_value) / self.position_start_value
        norm_profit = np.clip(position_profit, -1, 1)
        
        return np.array([
            self.position,  # Already normalized (-1 to 1)
            norm_spread,
            z_score,  # Normalize z-score
            norm_duration,
            norm_profit
        ])

    def _calculate_z_score(self, spread):
        window = self.spread.iloc[self.current_step-self.window_size:self.current_step]
        return (spread - window.mean()) / window.std()

    def _calculate_portfolio_reward(self):
        if self.position_start_value is None:
            return 0
        return (self.portfolio_value - self.position_start_value) / self.position_start_value

    def _calculate_immediate_reward(self):
        asset1_return = self.asset_returns_raw[f'{self.asset1} Return'].iloc[self.current_step]
        asset2_return = self.asset_returns_raw[f'{self.asset2} Return'].iloc[self.current_step]
        prev_index = self.asset_returns_raw.index[self.current_step-1]
        w2_prev = self.w2.loc[prev_index]
        
        spread_return = asset1_return - w2_prev * asset2_return

        
        return self.previous_position * spread_return
    
    def _get_baseline_action(self):
        z_score = self._calculate_z_score(self.spread.iloc[self.current_step])
        lower_threshold = -1 * self.upper_threshold
        
        if z_score > self.upper_threshold and self.position < 0:  # short signal
            return -1
        elif z_score < lower_threshold and self.position > 0:  # long signal
            return 1
        elif abs(z_score) < self.close_threshold and self.position == 0:  # neutral
            return 0
        
        return self.position  
    

In [135]:
def train_multi_pair_dqn(env, episodes, active_func, batch_size=32):
    state_size = 5  # position, spread, z-score, remaining_steps, portfolio_value
    action_size = len(env.action_space)
    
    agent = DQNAgent(state_size, action_size, active_func)
    
    # Initialize training metrics for each pair
    training_metrics = {
        # 'episode_rewards': [],
        # 'portfolio_values': [],
        'pair_performance': {pair: {
            'rewards': [],
            'portfolio_values': [],
            'best_portfolio_value': float('-inf'),
            'best_reward': float('-inf'),
            'best_cumulative_return':[],
            'best_state_dict': None
        } for pair in env.pair_list}
    }

    for episode in range(episodes):
        state = env.reset()
        current_pair = env.current_pair
        episode_reward = 0
        done = False

        while not done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            
            agent.memory.push(state, action, reward, next_state, done)
            if len(agent.memory) > batch_size:
                agent.learn()
            
            episode_reward += reward
            state = next_state
    
        
        # Store pair-specific metrics
        pair_metrics = training_metrics['pair_performance'][current_pair]
        pair_metrics['rewards'].append(episode_reward)
        pair_metrics['portfolio_values'].append(env.portfolio_value)
        
        # Update best model for current pair if performance improves
        if episode_reward > pair_metrics['best_reward']:
            pair_metrics['best_portfolio_value'] = env.portfolio_value
            pair_metrics['best_reward'] = episode_reward
            pair_metrics['best_state_dict'] = agent.policy_net.state_dict()
            pair_metrics['best_cumulative_return'] = env.cumulative_return
            
            # Update target network with best model
            agent.target_net.load_state_dict(pair_metrics['best_state_dict'])
            # print(f"\nNew best model found for pair {current_pair}!")
            # print(f"Episode {episode}")
            # print(f"Portfolio Value: {env.portfolio_value:.2f}")
            # print(f"Episode Reward: {episode_reward:.2f}")
        
        agent.decay_epsilon()

        # Print progress every 10 episodes
        # if episode % 10 == 0:
            
        #     print(f"\nEpisode {episode}")

            
                        
    # Find the best performing model across all pairs
    best_pair = max(training_metrics['pair_performance'].items(),
                   key=lambda x: x[1]['best_reward'])[0]
    best_metrics = training_metrics['pair_performance'][best_pair]
    
    # print(f"\nTraining complete! Best performing pair: {best_pair}")
    # print(f"Best portfolio value: {best_metrics['best_portfolio_value']:.2f}")
    # print(f"Best reward: {best_metrics['best_reward']:.2f}")
    
    # Set the final policy network to the best found
    agent.policy_net.load_state_dict(best_metrics['best_state_dict'])
    agent.target_net.load_state_dict(best_metrics['best_state_dict'])
    
    return agent, training_metrics

def backtest(env, agent,target_pair):
    state = env.reset(True, target_pair)
    done = False
    while not done:
        action = agent.act(state)
        state, reward, done, _ = env.step(action)
    
    return env.action_history

In [136]:

def plot_cumulative_returns(dates, training_returns, pair, type):

    plt.figure(figsize=(12, 6))
    plt.plot(dates, training_returns, label='Returns')
    plt.title(f'Cumulative Returns for {pair[0]} and {pair[1]} in {type} set')
    plt.xlabel('Dates')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def get_last_cumulative_return(dates, action_history, target_pair):

    pair_actions = [action for action in action_history if action['pair'] == target_pair]
    
    if pair_actions:
        last_action = pair_actions[-1]
        cumulative_return = last_action['cumulative_return']
        # plot_cumulative_returns(dates, cumulative_return, target_pair, 'Testing')
        
        mean_return, sharpe = get_summary(cumulative_return)

        return {
        'Mean Return (%) Annual': "{:.4f}".format(mean_return),
        'Sharpe Ratio Annual': "{:.4f}".format(sharpe)
    }
    else:
        return None
    
def get_summary(cumulative_return):

    periodic_returns = (1 + cumulative_return[1:]) / (1+cumulative_return[:-1]) - 1
    periodic_returns = periodic_returns[~np.isinf(periodic_returns) & ~np.isneginf(periodic_returns)]
    periodic_returns = periodic_returns[~np.isnan(periodic_returns)]
    # Mean return
    mean_return = np.mean(periodic_returns) *252 * 100
    
    # Sharpe ratio (assuming risk-free rate = 0)
    sharpe = (np.mean(periodic_returns) / np.std(periodic_returns))* np.sqrt(252)
    
    # Maximum drawdown
    cumulative = np.array(cumulative_return)
    running_max = np.maximum.accumulate(cumulative)

    # drawdowns = (cumulative - running_max) / running_max
    # drawdowns = drawdowns[~np.isnan(drawdowns)]
    # drawdowns = drawdowns[~np.isinf(drawdowns) & ~np.isneginf(drawdowns)]
    # max_drawdown = np.min(drawdowns) * 100  # Convert to percentage

    return mean_return, sharpe#, standard_error


In [140]:


if __name__ == "__main__":
    
    start_year = 2014
    end_year = 2023
    summary_data = []
    for year in range(start_year, end_year - 1):

        yearly_mask = pd.to_datetime(tr_dm_net.index).year == year
        yearly_plus1_mask = pd.to_datetime(tr_dm_net.index).year == year + 1
        yearly_plus2_mask = pd.to_datetime(tr_dm_net.index).year == year + 2

        formation_period = tr_dm_net[yearly_mask]
        train_period = tr_dm_net[yearly_plus1_mask]
        test_period = tr_dm_net[yearly_plus2_mask]

        formation_period_price = tr_dm_net_price[yearly_mask]
        train_period_price = tr_dm_net_price[yearly_plus1_mask]    

        noncoherent_pairs_in_formation = noncoherent_pair_cluster(formation_period, formation_period_price)

        window_size_list = [30, 60]
        upper_threshold_list = [1.0, 1.25, 1.5,1.75]
        close_threshold_list = [0.1, 0.2]

        # upper_threshold, close_threshold, window_size = optimize_parameters(train_period, noncoherent_pairs_in_formation, window_size_list, upper_threshold_list,close_threshold_list)
        upper_threshold = 1.0
        close_threshold = 0.1
        window_size = 60
        beta = 0.1

        env = MultiPairTradingEnv(train_period, noncoherent_pairs_in_formation, upper_threshold, close_threshold, window_size, beta)
        # optimal_active_fnc = test_activations(env)
        optimal_active_fnc = 'tanh'
        train_agent, train_env = train_multi_pair_dqn(env, episodes=100, active_func = optimal_active_fnc)
    
        training_dates = train_period.index[window_size+1:]
        for pair in noncoherent_pairs_in_formation:

            best_cumulative_return = train_env['pair_performance'][pair]['best_cumulative_return']
            # plot_cumulative_returns(training_dates, best_cumulative_return, pair, 'Training')
            mean_return, sharpe = get_summary(best_cumulative_return)

            summary_training = {'Mean Return (%) Annual': "{:.4f}".format(mean_return),
                                'Sharpe Ratio Annual': "{:.4f}".format(sharpe)
                                }
            
            summary_training['Pair'] = f"{pair[0]}-{pair[1]}"
            summary_training['Data Type'] = 'Training Data'
            summary_training['Period'] = f"{year} - {year + 2}"

            summary_data.append(summary_training)

        # Backtest
        noncoherent_pairs_in_train = noncoherent_pair_cluster(train_period, train_period_price)
        env_test = MultiPairTradingEnv(test_period, noncoherent_pairs_in_train, upper_threshold, close_threshold, window_size, beta)
        testing_dates = test_period.index[window_size+1:]
        for target_pair in noncoherent_pairs_in_train:

            backtest_actions = backtest(env_test, train_agent, target_pair)
            summary_testing = get_last_cumulative_return(testing_dates, backtest_actions, target_pair)
            summary_testing['Pair'] = f"{target_pair[0]}-{target_pair[1]}"
            summary_testing['Data Type'] = 'Test Data'
            summary_testing['Period'] = f"{year} - {year + 2}"

            summary_data.append(summary_testing)

    summary_df = pd.DataFrame(summary_data)
    
    # Reorder columns
    columns = ['Data Type', 'Period', 'Pair', 'Mean Return (%) Annual', 'Sharpe Ratio Annual']
    summary_df = summary_df[columns]
    


    


In [141]:
pivot_df = pd.pivot_table(
   summary_df,
   index=['Period', 'Pair'],
   columns='Data Type',
   values=['Mean Return (%) Annual', 'Sharpe Ratio Annual'],
   aggfunc='first' 
).round(4)
pivot_df.head(60)

Mean Return (%) Annual 
 Sharpe Ratio Annual 
 
 
 
 Data Type 
 Test Data 
 Training Data 
 Test Data 
 Training Data 
 
 
 Period 
 Pair 
 
 
 
 
 
 
 
 
 2014 - 2016 
 GT10 Govt Adjusted-GTCAD10Y Govt Adjusted 
 2.9862 
 NaN 
 0.8084 
 NaN 
 
 
 GT10 Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 10.1757 
 NaN 
 2.2503 
 
 
 GT10 Govt Adjusted-GTGBP10Y Govt Adjusted 
 3.5005 
 -4.9295 
 0.8517 
 -1.1394 
 
 
 GTCHF10Y Govt Adjusted-GT10 Govt Adjusted 
 2.9110 
 2.8007 
 0.8280 
 0.6927 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 0.0385 
 NaN 
 0.0130 
 NaN 
 
 
 GTCHF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 0.2850 
 NaN 
 0.1162 
 NaN 
 
 
 GTFRF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 -0.5869 
 NaN 
 -0.1461 
 
 
 GTITL10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 NaN 
 -0.6677 
 NaN 
 -0.0926 
 
 
 GTNZD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 NaN 
 1.3149 
 NaN 
 0.2833 
 
 
 2015 - 2017 
 GT10 Govt Adjusted-GTCAD10Y Govt Adjusted 
 -0.2669 
 2.0538 
 -0.0862 
 0.5868 
 
 
 GT10 Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 3.1958 
 NaN 
 0.8232 
 
 
 GTCHF10Y Govt Adjusted-GT10 Govt Adjusted 
 2.7960 
 -0.2967 
 1.0635 
 -0.0906 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 1.3107 
 4.3133 
 0.4661 
 1.5022 
 
 
 GTCHF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 -1.8872 
 NaN 
 -0.8488 
 
 
 GTFRF10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -3.8088 
 NaN 
 -1.4284 
 NaN 
 
 
 GTITL10Y Govt Adjusted-GTAUD10Y Govt Adjusted 
 -10.7915 
 NaN 
 -2.3219 
 NaN 
 
 
 GTITL10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 9.7541 
 NaN 
 2.0348 
 NaN 
 
 
 2016 - 2018 
 GT10 Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 -2.8837 
 NaN 
 -1.0757 
 
 
 GTCHF10Y Govt Adjusted-GT10 Govt Adjusted 
 NaN 
 0.2537 
 NaN 
 0.0999 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 -3.9826 
 NaN 
 -1.4375 
 
 
 GTFRF10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 -4.7842 
 NaN 
 -1.8514 
 
 
 GTFRF10Y Govt Adjusted-GTITL10Y Govt Adjusted 
 -1.5296 
 NaN 
 -0.2564 
 NaN 
 
 
 GTITL10Y Govt Adjusted-GTAUD10Y Govt Adjusted 
 NaN 
 -6.4109 
 NaN 
 -1.4047 
 
 
 GTITL10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 NaN 
 2.2631 
 NaN 
 0.4381 
 
 
 GTNZD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 -1.2335 
 NaN 
 -0.4092 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GT10 Govt Adjusted 
 -2.5503 
 NaN 
 -0.9434 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 -6.0246 
 NaN 
 -2.3097 
 NaN 
 
 
 2017 - 2019 
 GT10 Govt Adjusted-GTDEM10Y Govt Adjusted 
 -4.4216 
 NaN 
 -1.0066 
 NaN 
 
 
 GTCAD10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 7.0178 
 NaN 
 1.5552 
 NaN 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 3.1003 
 NaN 
 0.8822 
 NaN 
 
 
 GTFRF10Y Govt Adjusted-GTITL10Y Govt Adjusted 
 NaN 
 0.3757 
 NaN 
 0.0628 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 0.0496 
 NaN 
 0.0202 
 NaN 
 
 
 GTGBP10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 1.4412 
 NaN 
 0.4054 
 NaN 
 
 
 GTNZD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 NaN 
 -1.1982 
 NaN 
 -0.4057 
 
 
 GTSEK10Y Govt Adjusted-GT10 Govt Adjusted 
 NaN 
 1.3816 
 NaN 
 0.5464 
 
 
 GTSEK10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 0.9101 
 NaN 
 0.3679 
 
 
 2018 - 2020 
 GT10 Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 -1.9024 
 NaN 
 -0.4354 
 
 
 GTCAD10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 1.5661 
 NaN 
 0.5090 
 NaN 
 
 
 GTCAD10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 -1.1147 
 NaN 
 -0.2459 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 -4.3108 
 -1.2426 
 -1.4126 
 -0.3670 
 
 
 GTFRF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 2.5176 
 NaN 
 0.8604 
 NaN 
 
 
 GTFRF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 -2.4177 
 NaN 
 -0.8703 
 NaN 
 
 
 GTFRF10Y Govt Adjusted-GTSEK10Y Govt Adjusted 
 NaN 
 1.1793 
 NaN 
 0.4442 
 
 
 GTGBP10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 1.1526 
 -1.1897 
 0.2898 
 -0.3317 
 
 
 GTITL10Y Govt Adjusted-GTAUD10Y Govt Adjusted 
 -0.9364 
 NaN 
 -0.1403 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GTCAD10Y Govt Adju

In [142]:
pivot_df.tail(32)

Mean Return (%) Annual 
 Sharpe Ratio Annual 
 
 
 
 Data Type 
 Test Data 
 Training Data 
 Test Data 
 Training Data 
 
 
 Period 
 Pair 
 
 
 
 
 
 
 
 
 2019 - 2021 
 GTCAD10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 -0.7182 
 NaN 
 -0.2395 
 
 
 GTCHF10Y Govt Adjusted-GT10 Govt Adjusted 
 1.5543 
 NaN 
 0.4449 
 NaN 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 -5.4240 
 3.1682 
 -1.7611 
 1.0337 
 
 
 GTCHF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 -0.9927 
 NaN 
 -0.3923 
 NaN 
 
 
 GTFRF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 1.1829 
 NaN 
 0.3705 
 
 
 GTFRF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 -0.8553 
 NaN 
 -0.3971 
 
 
 GTGBP10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 0.8968 
 NaN 
 0.2303 
 
 
 GTITL10Y Govt Adjusted-GTAUD10Y Govt Adjusted 
 NaN 
 0.7043 
 NaN 
 0.1242 
 
 
 GTNZD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 -0.8124 
 NaN 
 -0.1940 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GT10 Govt Adjusted 
 -1.7910 
 NaN 
 -0.5892 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 -1.0424 
 -0.8089 
 -0.3637 
 -0.2570 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 -2.7799 
 NaN 
 -1.5925 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 2.9434 
 -1.6873 
 1.5142 
 -0.6903 
 
 
 GTSEK10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 1.4998 
 NaN 
 0.6649 
 NaN 
 
 
 2020 - 2022 
 GT10 Govt Adjusted-GTDEM10Y Govt Adjusted 
 9.0830 
 NaN 
 1.2439 
 NaN 
 
 
 GTAUD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 NaN 
 9.0278 
 NaN 
 1.9331 
 
 
 GTAUD10Y Govt Adjusted-GTNZD10Y Govt Adjusted 
 NaN 
 1.2932 
 NaN 
 0.2861 
 
 
 GTCAD10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 -6.6472 
 NaN 
 -1.1548 
 NaN 
 
 
 GTCHF10Y Govt Adjusted-GT10 Govt Adjusted 
 NaN 
 5.4053 
 NaN 
 1.6654 
 
 
 GTCHF10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 1.5478 
 NaN 
 0.5468 
 
 
 GTCHF10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 1.6591 
 NaN 
 0.7190 
 
 
 GTNZD10Y Govt Adjusted-GTJPY10Y Govt Adjusted 
 NaN 
 5.0598 
 NaN 
 1.3587 
 
 
 GTSEK10Y Govt Adjusted-GT10 Govt Adjusted 
 -5.8501 
 2.0755 
 -0.7873 
 0.7056 
 
 
 GTSEK10Y Govt Adjusted-GTCAD10Y Govt Adjusted 
 NaN 
 1.2729 
 NaN 
 0.4453 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 NaN 
 1.1241 
 NaN 
 0.6829 
 
 
 GTSEK10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 -0.4985 
 NaN 
 -0.2758 
 
 
 GTSEK10Y Govt Adjusted-GTGBP10Y Govt Adjusted 
 NaN 
 -0.7138 
 NaN 
 -0.3217 
 
 
 2021 - 2023 
 GT10 Govt Adjusted-GTDEM10Y Govt Adjusted 
 3.6072 
 3.0667 
 0.5266 
 0.4255 
 
 
 GTCAD10Y Govt Adjusted-GTDEM10Y Govt Adjusted 
 NaN 
 1.5358 
 NaN 
 0.2420 
 
 
 GTFRF10Y Govt Adjusted-GT10 Govt Adjusted 
 -2.0499 
 NaN 
 -0.3766 
 NaN 
 
 
 GTSEK10Y Govt Adjusted-GT10 Govt Adjusted 
 NaN 
 5.6018 
 NaN 
 0.6820 
 
 
 GTSEK10Y Govt Adjusted-GTCHF10Y Govt Adjusted 
 0.6354 
 NaN 
 0.1335 
 NaN